In [6]:
from tensorflow import keras

model = keras.models.load_model("facenet_keras.h5", compile=False)

In [ ]:
import numpy as np
import os, time
import cv2
from skimage.transform import resize
from scipy.spatial import distance

#驗證compares列表中相片的人與valid相片中的人
valid = "200127/200127_1.jpg"
compares = ["200002/200002_1.jpg", "200127/200127_1", "200127/o.jpg" ]

#用OpenCV的Cascade classifier來偵測臉部，不一定跟Facenet一樣要用MTCNN。
cascade_path = "haarcascade_frontalface_default.xml"

#我們的人像相片都放置於validPicPath
validPicPath = "members/"

#此版Facenet model需要的相片尺寸為160×160
image_size = 160

In [ ]:
#圖像白化（whitening）可用於對過度曝光或低曝光的圖片進行處理，處理的方式就是改變圖像的平均像素值為 0 ，改變圖像的方差為單位方差 1。
def prewhiten(x):

    if x.ndim == 4:
        axis = (1, 2, 3)
        size = x[0].size

    elif x.ndim == 3:
        axis = (0, 1, 2)
        size = x.size

    else:
        raise ValueError("Dimension should be 3 or 4")
    mean = np.mean(x, axis=axis, keepdims=True)
    std = np.std(x, axis=axis, keepdims=True)
    std_adj = np.maximum(std, 1.0/np.sqrt(size))
    y = (x – mean) / std_adj

    return y

In [ ]:
#使用L1或L2標準化圖像，可強化其特徵。
def l2_normalize(x, axis=-1, epsilon=1e-10):

    output = x / np.sqrt(np.maximum(np.sum(np.square(x), axis=axis, keepdims=True), epsilon))

    return output

In [ ]:
#偵測並取得臉孔area，接著再resize為模型要求的尺寸(下方例子並未作alignment)
def align_image(img, margin):

    cascade = cv2.CascadeClassifier(cascade_path)
    faces = cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=3)

    if(len(faces)>0):
        (x, y, w, h) = faces[0]
        face = img[y:y+h, x:x+w]
        faceMargin = np.zeros((h+margin*2, w+margin*2, 3), dtype = “uint8″)
        faceMargin[margin:margin+h, margin:margin+w] = face
        cv2.imwrite(str(time.time())+".jpg", faceMargin)
        aligned = resize(faceMargin, (image_size, image_size), mode=’reflect’)
        cv2.imwrite(str(time.time())+"_aligned.jpg", aligned)

        return aligned

    else:
        return None

In [ ]:
#圖像的預處理(即前述的幾項步驟)
def preProcess(img):
    whitenImg = prewhiten(img)
    whitenImg = whitenImg[np.newaxis, :]
    return whitenImg

imgValid = validPicPath + valid
aligned = align_image(cv2.imread(imgValid), 6)

if(aligned is None):
    print("Cannot find any face in image: {}".format(imgValid))

else:
    faceImg = preProcess(aligned)
    
    #–> model會輸出128維度的臉孔特徵向量，接著我們將它們合併並進行L2正規化。Z
    embs_valid = l2_normalize(np.concatenate(model.predict(faceImg)))

    #同上方的valid圖片，依序取得各圖片人臉的臉孔特徵向量，再與valid進行歐氏距離計算。
    for member in compares:

        img_file = validPicPath + member
        aligned = align_image(cv2.imread(img_file), 6)

        if(aligned is not None):
            faceImg = preProcess(aligned)
            embs = l2_normalize(np.concatenate(model.predict(faceImg)))
            distanceNum = distance.euclidean(embs_valid, embs)
            print("Diff with {} is {}".format(member, distanceNum))